# Data Cleaning - Sentiment analysis

## Load Data

In [15]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rdolphin/financial-news-with-ticker-level-sentiment")

print("Path to dataset files:", path)

Path to dataset files: /Users/hippolyteheger/.cache/kagglehub/datasets/rdolphin/financial-news-with-ticker-level-sentiment/versions/1


In [16]:
import os
import pandas as pd

print(os.listdir(path))

['polygon_news_sample.json']


In [17]:
import pandas as pd
import json

file_path = f"{path}/polygon_news_sample.json"

with open(file_path, "r") as f:
    data = json.load(f)

df = pd.DataFrame(data)
df.head()

,article_url,author,description,id,image_url,amp_url,keywords,published_utc,publisher,tickers,insights,title
0,https://www.zacks.com/stock/news/2114673/alleg...,Zacks.com,Allegiant Travel (ALGT) is a fast-moving stock...,db221630f08d9064b6539534cc9957ecd7ba2a626927c7...,https://staticx-tuner.zacks.com/images/default...,None,"[Allegiant Travel, momentum investing, fast-pa...",2023-06-29T12:50:06Z,"{'name': 'Zacks Investment Research', 'homepag...",[ALGT],"[{'ticker': 'ALGT', 'sentiment': 'positive', '...",Allegiant Travel (ALGT) Is Attractively Priced...
1,https://www.zacks.com/stock/news/2085677/appli...,Zacks.com,Applied Industrial Technologies (AIT) reported...,bb7e1725949a7254ae18e8d149c3c19af050c0ac05f18f...,https://staticx-tuner.zacks.com/images/default...,None,"[earnings, revenues, estimates, industrial pro...",2023-04-27T11:55:14Z,"{'name': 'Zacks Investment Research', 'homepag...","[AIT, NPO]","[{'ticker': 'AIT', 'sentiment': 'positive', 's...",Applied Industrial Technologies (AIT) Q3 Earni...
2,https://www.globenewswire.com/news-release/202...,N/A,"Apollo Commercial Real Estate Finance, Inc. (A...",a49c53ef44092946950dfb3f33852c9ef07d7c7dc6c1ea...,https://ml.globenewswire.com/Resource/Download...,None,"[commercial real estate, financing, mortgage l...",2023-03-06T13:30:00Z,"{'name': 'GlobeNewswire Inc.', 'homepage_url':...","[ARI, SAN]","[{'ticker': 'ARI', 'sentiment': 'positive', 's...","Apollo Commercial Real Estate Finance, Inc. Cl..."
3,https://www.globenewswire.com/news-release/202...,N/A,"Maravai LifeSciences, a global provider of lif...",be4f5174307cd0f3309ee931ab4ec4fc2451af056769ca...,https://ml.globenewswire.com/Resource/Download...,None,"[Maravai LifeSciences, investor conferences, f...",2023-11-09T13:15:00Z,"{'name': 'GlobeNewswire Inc.', 'homepage_url':...",[MRVI],"[{'ticker': 'MRVI', 'sentiment': 'positive', '...",Maravai LifeSciences Announces November 2023 I...
4,https://www.zacks.com/stock/news/2069321/dht-h...,Zacks Equity Research,"DHT Holdings, an independent oil tanker compan...",29bea2bb15df75a10fd940c2dc705d21d4c413fb45c17a...,https://staticx-tuner.zacks.com/images/default...,None,"[DHT Holdings, oil tanker, earnings, revenue, ...",2023-03-22T22:00:25Z,"{'name': 'Zacks Investment Research', 'homepag...",[DHT],"[{'ticker': 'DHT', 'sentiment': 'neutral', 'se...",DHT Holdings (DHT) Stock Moves -1.33%: What Yo...


In [18]:
df.columns

Index(['article_url', 'author', 'description', 'id', 'image_url', 'amp_url',
       'keywords', 'published_utc', 'publisher', 'tickers', 'insights',
       'title'],
      dtype='object')

In [19]:
df.dtypes

article_url      object
author           object
description      object
id               object
image_url        object
amp_url          object
keywords         object
published_utc    object
publisher        object
tickers          object
insights         object
title            object
dtype: object

## Data Cleaning

In [20]:
df[['sentiment', 'sentiment_reasoning']] = df['insights'].apply(lambda x: pd.Series({
    'sentiment': x[0]['sentiment'],
    'sentiment_reasoning': x[0]['sentiment_reasoning']
}))

In [21]:
df = df[['description', 'sentiment','sentiment_reasoning',
       'title']]

In [22]:
#df['published_utc'] = pd.to_datetime(df['published_utc']).dt.date
1#df = df.set_index('published_utc')


1

In [ ]:
def extract_sentiments(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts sentiment and sentiment_reasoning from the 'insights' column,
    and returns a DataFrame with only the relevant columns:
    ['description', 'sentiment', 'sentiment_reasoning', 'title'].

    Args:
        df (pd.DataFrame): Original DataFrame with 'insights' column.

    Returns:
        pd.DataFrame: Processed DataFrame with extracted sentiment info.
    """
    # Extract 'sentiment' and 'sentiment_reasoning' from insights
    df[['sentiment', 'sentiment_reasoning']] = df['insights'].apply(
        lambda x: pd.Series({
            'sentiment': x[0]['sentiment'],
            'sentiment_reasoning': x[0]['sentiment_reasoning']
        })
    )

    # Keep only the relevant columns
    df = df[['description', 'sentiment', 'sentiment_reasoning', 'title']]

    return df

## API Fetching

In [ ]:
import requests
import pandas as pd
from datetime import datetime

def fetch_massive_news_df(api_key: str, start_date: str, end_date: str = None, order: str = "desc") -> pd.DataFrame:
    """
    Fetch news from Massive API for a specific date range and return as a pandas DataFrame.

    Args:
        api_key (str): Your Massive API key.
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str, optional): End date in 'YYYY-MM-DD' format. Defaults to today.
        order (str, optional): 'asc' or 'desc' for sorting by published date. Default is 'desc'.

    Returns:
        pd.DataFrame: DataFrame with news articles.
    """
    url = "https://api.massive.com/v2/reference/news"

    # Convert end_date to today if not provided
    if end_date is None:
        end_date = datetime.utcnow().strftime("%Y-%m-%d")

    # Convert to ISO format for the API
    start_iso = datetime.strptime(start_date, "%Y-%m-%d").strftime("%Y-%m-%dT%H:%M:%SZ")
    end_iso = datetime.strptime(end_date, "%Y-%m-%d").strftime("%Y-%m-%dT%H:%M:%SZ")

    params = {
        "order": order,
        "sort": "published_utc",
        "limit": 1000,  # Max per request
        "start_date": start_iso,
        "end_date": end_iso,
        "apiKey": api_key
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()

        # Convert to DataFrame
        if "results" in data:
            df = pd.DataFrame(data["results"])
            return df
        else:
            print("No results found in the API response.")
            return pd.DataFrame()

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame()


    return df_news

# Example usage:
api_key = "SiV7GQdKTF2ZtrAr1xNSrnNYP11dKCAC"
df_news = fetch_massive_news_df(api_key, start_date="2025-11-03", end_date="2025-12-03")
# print(df_news.head())

In [84]:
df_news.head()

,id,publisher,title,author,published_utc,article_url,tickers,image_url,description,keywords,insights,amp_url
0,2e91348698e0f7dcc317815dab057aa49810638f3cb7b9...,"{'name': 'The Motley Fool', 'homepage_url': 'h...",Why GitLab Stock Crashed Today,Rich Smith,2025-12-03T15:27:35Z,https://www.fool.com/investing/2025/12/03/why-...,[GTLB],https://g.foolcdn.com/image/?url=https%3A%2F%2...,GitLab reported strong Q3 earnings with 25% ye...,"[GitLab, earnings, stock, revenue, guidance]","[{'ticker': 'GTLB', 'sentiment': 'neutral', 's...",NaN
1,36da3834e8dd32a195821a7e640d79398b2e97c3b117cd...,"{'name': 'The Motley Fool', 'homepage_url': 'h...",My Top 5 Stocks to Buy Now in December (2025),"Parkev Tatevosian, Cfa",2025-12-03T15:27:26Z,https://www.fool.com/investing/2025/12/03/my-t...,"[NVDA, AMZN, LLY, UBER, PINS]",https://g.foolcdn.com/image/?url=https%3A%2F%2...,The article highlights top stock recommendatio...,"[stocks, investment, technology, consumer good...","[{'ticker': 'NVDA', 'sentiment': 'positive', '...",NaN
2,5cff0820995fe4b49d8d02501450b233d6b755cec9c7f9...,"{'name': 'The Motley Fool', 'homepage_url': 'h...",Walmart vs. Costco: What's the Better Retail S...,David Jagielski,2025-12-03T15:25:00Z,https://www.fool.com/investing/2025/12/03/walm...,"[WMT, COST]",https://g.foolcdn.com/image/?url=https%3A%2F%2...,The article compares Walmart and Costco as ret...,"[retail stocks, investment, stock comparison, ...","[{'ticker': 'WMT', 'sentiment': 'positive', 's...",NaN
3,86d56402e4c2fa98f0838a07ec69dd781dde7876b253f6...,"{'name': 'The Motley Fool', 'homepage_url': 'h...",Why Did CoreWeave Stock Plunge 45% in November?,Howard Smith,2025-12-03T15:23:56Z,https://www.fool.com/investing/2025/12/03/why-...,"[CRWV, NVDA]",https://g.foolcdn.com/image/?url=https%3A%2F%2...,CoreWeave's stock dropped 45% in November due ...,"[AI, cloud computing, stock market, technology...","[{'ticker': 'CRWV', 'sentiment': 'neutral', 's...",NaN
4,2afcd9f9abcbb4de421d24c9b219902595f4c0641576d6...,"{'name': 'Investing.com', 'homepage_url': 'htt...",Salesforce Earnings Preview: Can It Finally Mo...,Ali Merchant,2025-12-03T15:18:00Z,https://www.investing.com/analysis/salesforce-...,[CRM],https://i-invdn-com.investing.com/redesign/ima...,Salesforce is expected to announce an 8.8% rev...,"[AI, earnings, revenue growth, acquisitions, s...","[{'ticker': 'CRM', 'sentiment': 'neutral', 'se...",https://m.investing.com/analysis/salesforce-ea...


In [85]:
def filter_news_by_tickers(df_news: pd.DataFrame, tickers: list) -> pd.DataFrame:
    """
    Filter a Massive API news DataFrame by tickers.

    Args:
        df_news (pd.DataFrame): DataFrame returned by fetch_massive_news_df.
        tickers (list): List of tickers to filter (e.g., ["AAPL", "MSFT"]).

    Returns:
        pd.DataFrame: Filtered DataFrame containing only news for the specified tickers.
    """
    # If df is empty → return empty df
    if df_news.empty:
        return df_news

    # Ensure tickers column exists
    if "tickers" not in df_news.columns:
        print("Warning: 'tickers' column missing in the data.")
        return pd.DataFrame()

    # Normalize tickers (case-insensitive)
    tickers = [t.upper() for t in tickers]

    # Filter rows containing at least one of the tickers
    filtered_df = df_news[
        df_news["tickers"].apply(
            lambda lst: any(t in lst for t in tickers) if isinstance(lst, list) else False
        )
    ]

    return filtered_df

In [86]:
df_filtered = filter_news_by_tickers(df_news,["LLY"])
df_filtered

,id,publisher,title,author,published_utc,article_url,tickers,image_url,description,keywords,insights,amp_url
1,36da3834e8dd32a195821a7e640d79398b2e97c3b117cd...,"{'name': 'The Motley Fool', 'homepage_url': 'h...",My Top 5 Stocks to Buy Now in December (2025),"Parkev Tatevosian, Cfa",2025-12-03T15:27:26Z,https://www.fool.com/investing/2025/12/03/my-t...,"[NVDA, AMZN, LLY, UBER, PINS]",https://g.foolcdn.com/image/?url=https%3A%2F%2...,The article highlights top stock recommendatio...,"[stocks, investment, technology, consumer good...","[{'ticker': 'NVDA', 'sentiment': 'positive', '...",NaN
358,b45f773db422f56f1d1f020125d05fd3493d4a861aafd8...,"{'name': 'Benzinga', 'homepage_url': 'https://...",History Says These 7 S&P 500 Stocks Often Rall...,Piero Cingari,2025-12-01T21:33:48Z,https://www.benzinga.com/markets/equities/25/1...,"[CCL, RCL, WDC, STX, MHK, LLY, GS, GSpA, GSpC,...",https://cdn.benzinga.com/files/images/story/20...,Historical data suggests certain S&P 500 stock...,"[December rally, seasonal stocks, S&P 500, mar...","[{'ticker': 'CCL', 'sentiment': 'positive', 's...",NaN
372,388eb3161333e7ded5feceb2a40b6d7fb030caa8376073...,"{'name': 'Benzinga', 'homepage_url': 'https://...",Zepbound Just Got Cheaper: Lilly Cuts Prices O...,Vandana Singh,2025-12-01T20:41:17Z,https://www.benzinga.com/markets/large-cap/25/...,"[LLY, NVO]",https://cdn.benzinga.com/files/images/story/20...,Eli Lilly reduced prices for Zepbound single-d...,"[Zepbound, weight loss, medication, price redu...","[{'ticker': 'LLY', 'sentiment': 'positive', 's...",NaN
397,c903db4813b9d96baa7fa93013bce9fbeb003a446dab59...,"{'name': 'Investing.com', 'homepage_url': 'htt...",AI Stocks Expose Market Vulnerability as Overv...,Dave Kovaleski,2025-12-01T18:08:00Z,https://www.investing.com/analysis/ai-stocks-e...,"[NVDA, ARM, AMD, PLTR, LLY]",https://i-invdn-com.investing.com/redesign/ima...,November saw technology and AI stocks experien...,"[AI stocks, market valuation, tech stocks, Nov...","[{'ticker': 'NVDA', 'sentiment': 'negative', '...",https://m.investing.com/analysis/ai-stocks-exp...
813,3750cd94c3619ad40ad3a11f3c05e84e3158f9b6e3294f...,"{'name': 'GlobeNewswire Inc.', 'homepage_url':...",Atopic Dermatitis Drugs Market to Reach $29.43...,Sns Insider,2025-11-28T05:28:00Z,https://www.globenewswire.com/news-release/202...,"[SNY, REGN, ABBV, PFE, LLY]",https://ml.globenewswire.com/Resource/Download...,The global Atopic Dermatitis Drugs Market is p...,"[atopic dermatitis, pharmaceutical market, bio...","[{'ticker': 'SNY', 'sentiment': 'positive', 's...",NaN
866,6822b8caefe074fcde24fc851c4969767ab64169c8fd74...,"{'name': 'GlobeNewswire Inc.', 'homepage_url':...",Human Growth Hormone Injection Market Size Exp...,Towards Healthcare,2025-11-27T15:00:00Z,https://www.globenewswire.com/news-release/202...,"[PFE, NVO, LLY]",https://ml.globenewswire.com/Resource/Download...,The human growth hormone injection market is e...,"[human growth hormone, HGH, medical market, bi...","[{'ticker': 'PFE', 'sentiment': 'positive', 's...",NaN
882,9c989bb899500b8138b74d066c938fa08176b1f8575fec...,"{'name': 'The Motley Fool', 'homepage_url': 'h...",Prediction: This Undervalued Stock Could Compe...,Prosper Junior Bakiny,2025-11-27T13:15:00Z,https://www.fool.com/investing/2025/11/27/pred...,"[VKTX, NVO, LLY]",https://g.foolcdn.com/image/?url=https%3A%2F%2...,Viking Therapeutics is developing a promising ...,"[weight loss, biotech, pharmaceutical, clinica...","[{'ticker': 'VKTX', 'sentiment': 'positive', '...",NaN
888,90a3dddcde1cfc0f245432ba60857aad44dcc1085fadde...,"{'name': 'GlobeNewswire Inc.', 'homepage_url':...",Anti-Obesity Drugs Market Size to Reach $35.52...,Sns Insider,2025-11-27T12:54:00Z,https://www.globenewswire.com/news-release/202...,"[NVO, LLY, PFE]",https://ml.globenewswire.com/Resource/Download...,The global anti-obesity drugs market is projec...,"[anti-obesity drugs, weight management, GLP-1 ...","[{'ticker': 'NVO', 'sentiment': 'positive', 's...",NaN
892,6862d31d67ea299d521af2

In [88]:
def explode_insights(df: pd.DataFrame) -> pd.DataFrame:
    """
    Takes the filtered news DataFrame and explodes each insight entry
    into separate rows with: date, ticker, title, sentiment, reasoning.

    Returns:
        pd.DataFrame: A transformed DataFrame ready for RL pipelines.
    """

    all_rows = []

    for _, row in df.iterrows():
        date = row["published_utc"]
        title = row.get("title", None)
        insights = row.get("insights", [])

        # Skip rows without insights
        if not isinstance(insights, list):
            continue

        for item in insights:
            ticker = item.get("ticker")
            sentiment = item.get("sentiment")
            reasoning = item.get("sentiment_reasoning")

            all_rows.append({
                "date": date,
                "ticker": ticker,
                "title": title,
                "sentiment": sentiment,
                "sentiment_reasoning": reasoning
            })

    # Convert to DataFrame
    df_out = pd.DataFrame(all_rows)

    # Index by date
    if not df_out.empty:
        df_out["date"] = pd.to_datetime(df_out["date"])
        df_out = df_out.set_index("date").sort_index()

    return df_out

In [ ]:
df_filtered = filter_news_by_tickers(df_news, ["AAPL", "MSFT", "NVDA"])

df_rl = explode_insights(df_filtered)

df_rl.head()


,ticker,title,sentiment,sentiment_reasoning
date,,,,
2025-11-26 19:15:00+00:00,NOK,1 Unstoppable Artificial Intelligence (AI) Sto...,neutral,Strategic investment target for Nvidia's AI te...
2025-11-26 19:15:00+00:00,NVDA,1 Unstoppable Artificial Intelligence (AI) Sto...,positive,"Projected 360% potential growth, dominant posi..."
2025-11-26 19:15:00+00:00,MSFT,1 Unstoppable Artificial Intelligence (AI) Sto...,neutral,Part of hyperscalers expected to invest heavil...
2025-11-26 19:15:00+00:00,AMZN,1 Unstoppable Artificial Intelligence (AI) Sto...,neutral,Participating in AI infrastructure investments...
2025-11-26 19:32:00+00:00,NVDA,TPUs or GPUs: Is Nvidia’s Moat Eroding?,neutral,While facing potential competition from custom...


df_rl